In [1]:
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc


ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmdb3 libmdbsql3
Suggested packages:
  bash-completion
The following NEW packages will be installed:
  libmdb3 libmdbsql3 mdbtools
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 136 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdb3 amd64 1.0.0+dfsg-1 [68.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdbsql3 amd64 1.0.0+dfsg-1 [18.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mdbtools amd64 1.0.0+dfsg-1 [48.3 kB]
Fetched 136 kB in 0s (729 kB/s)
Selecting previously unselected package libmdb3:amd64.
(Reading databas

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util


In [ ]:

# 使用例
table_name = 'PredictorVariables'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]
years = range(2000, 2025)

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

df_statistics=[]
for y in years:
  filepath = f'/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb'
  df_summary = k_a_util.access_file_statistics(filepath, table_name)
  columns_to_drop = [ 'key_horse_id', 'desc_horse_no', 'upd', 'obj_rank']
  df = df_summary.drop(columns=columns_to_drop)
  df_statistics.append(df)
  # print(file_name)
  # print(df)
  # print("")

In [12]:
import pandas as pd
import numpy as np

def summarize_dataframes(dfs, years):
    """
    複数のDataFrameの統計量を計算し、年ごとにまとめたDataFrameを作成する。

    Args:
        dfs: DataFrameのリスト
        years: 年のリスト (dfsと同じ長さであること)

    Returns:
        統計量をまとめたDataFrame。DataFrameのリストと年のリストの長さが異なる場合はNoneを返す。
        DataFrameが空の場合もNoneを返す。
    """
    if not dfs or not all(not df.empty for df in dfs):
        print("Error: Empty DataFrame(s) provided.")
        return None

    if len(dfs) != len(years):
        print("Error: Length of DataFrames and years lists must be the same.")
        return None

    all_numeric_cols = set()
    for df in dfs:
        numeric_cols = df.select_dtypes(include=np.number).columns
        all_numeric_cols.update(numeric_cols)

    summary = pd.DataFrame(index=years)

    for col in all_numeric_cols:
        for i, (df, year) in enumerate(zip(dfs, years)):
            if col in df.columns:  # DataFrameに列が存在するか確認
                summary.loc[year, (col, 'mean')] = df[col].mean()
                summary.loc[year, (col, 'std')] = df[col].std()
                summary.loc[year, (col, 'min')] = df[col].min()
            else:
                # 列が存在しない場合はNaNを設定して警告を出す
                print(f"Warning: Column '{col}' not found in DataFrame for year {year}.")
                summary.loc[year, (col, 'mean')] = np.nan
                summary.loc[year, (col, 'std')] = np.nan
                summary.loc[year, (col, 'min')] = np.nan

    return summary




      cats_plc   mean    std    min  obj_rank
2000    0.0000 3.5961 3.4052 0.0000    0.0000
2001    0.0000 3.6620 3.4686 0.0000    0.0000

CSVファイルが見つかりませんでした。
上記はサンプルデータを用いた実行結果です。


In [11]:
# CSVファイルから読み込む場合の例 (3つのファイルを読み込む例に変更)
try:
    summary_df_from_csv = summarize_dataframes(dfs_csv, years_csv)
    print("\nCSVから読み込んだ場合の例:")
    print(summary_df_from_csv)

except FileNotFoundError as e:
    print(f"\nCSVファイルが見つかりませんでした: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except pd.errors.EmptyDataError as e:
    print(f"\nCSVファイルが空です: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except Exception as e:  # その他の例外をキャッチ
    print(f"\nCSVファイルの処理中にエラーが発生しました: {e}")
    print("上記はサンプルデータを用いた実行結果です。")

# エラーが発生するケースのテスト